In [1]:
#if in aws
from io import StringIO
from zipfile import Path
zipped = Path("/home/ubuntu/work/BERT_Family/data/news/news.zip", at="train.csv")

In [2]:
import pandas as pd #for read data
import numpy as np
def load_data_to_demoData(data = None, sample_size = 2000):
    sample_size = sample_size
    demo_data = pd.DataFrame()
    for i in pd.unique(data.label):
        population = data[data.label == i].index
        demo_data = demo_data.append(data.iloc[np.random.choice(population, sample_size, replace=False), :])
    return demo_data

建立Dataset

In [3]:
from torch.utils.data import Dataset
import torch
from datasets import load_dataset
#demo_data = pd.DataFrame(load_dataset('glue', 'cola', split='train'))
class MyDemoDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        #data = pd.read_csv("~/Desktop/train.csv") #in mac
        data = pd.read_csv(StringIO(zipped.read_text()))#in aws
        demo_data = load_data_to_demoData(data=data)
        self.x = demo_data[["title1_zh", "title1_zh"]]
        self.y = demo_data["label"]
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        #無意義處理，純粹示範用
        result = len(self.x.iloc[index, 0])
        return result

建立Dataloader

In [4]:
from torch.utils.data import DataLoader
myDataLoader = DataLoader(dataset=MyDemoDataset(), shuffle=True, batch_size=5)

/tmp/ipykernel_2301/168849740.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  demo_data = demo_data.append(data.iloc[np.random.choice(population, sample_size, replace=False), :])
/tmp/ipykernel_2301/168849740.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  demo_data = demo_data.append(data.iloc[np.random.choice(population, sample_size, replace=False), :])
/tmp/ipykernel_2301/168849740.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  demo_data = demo_data.append(data.iloc[np.random.choice(population, sample_size, replace=False), :])


In [82]:

for i, data in enumerate(myDataLoader):
    print(data)
    if i == 3:
        break 
    
demo_iter = iter(myDataLoader)
next(demo_iter)

tensor([21, 22, 28, 19, 27])
tensor([24, 30, 11, 20, 22])
tensor([29, 16, 17, 21, 24])
tensor([30, 23, 22, 23, 27])


tensor([30, 30, 21, 30, 30])

Transformers示範:
Pre-trained Tokenizer from BERT Model

In [5]:
from transformers import BertTokenizer, AlbertForSequenceClassification
MY_PRETRAINED_MODEL = "uer/albert-base-chinese-cluecorpussmall"
exmaple_text = "為了DEMO而示範用的句子，看看這句Tokenize後成效如何。老鼠和老虎傻傻分不清楚"
tokenizer = BertTokenizer.from_pretrained(MY_PRETRAINED_MODEL)
print(tokenizer(exmaple_text))

{'input_ids': [101, 4158, 749, 100, 5445, 4850, 5061, 4500, 4638, 1368, 2094, 8024, 4692, 4692, 6857, 1368, 100, 2527, 2768, 3126, 1963, 862, 511, 5439, 7962, 1469, 5439, 5988, 1004, 1004, 1146, 679, 3926, 3504, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Modify __getitem__() on MyDemoDataset:

In [6]:
MAX_LENGTH = 80
class MyDemoDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        #data = pd.read_csv("~/Desktop/train.csv") #in mac
        data = pd.read_csv(StringIO(zipped.read_text()))#in aws
        demo_data = load_data_to_demoData(data=data)
        self.x = demo_data[["title1_zh", "title2_zh"]]
        self.y = demo_data["label"]
        self.y_dict = {ele:i for i, ele in enumerate(pd.unique(self.y))}
        self.tokenizer = BertTokenizer.from_pretrained(MY_PRETRAINED_MODEL)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        tokenize_result = self.tokenizer(self.x.iloc[index, 0], self.x.iloc[index, 1],
                                         padding="max_length", truncation=True, 
                                         max_length=MAX_LENGTH, return_tensors="pt")
        input_ids, token_type_ids, attention_mask = map(torch.squeeze, tokenize_result.values())
        mylabel =self.y.iloc[index,]
        return input_ids, token_type_ids, attention_mask, self.y_dict[mylabel]

#測試看看如何
myDataLoader = DataLoader(dataset=MyDemoDataset(), shuffle=True, batch_size=128, pin_memory=True)
print(next(iter(myDataLoader)))

/tmp/ipykernel_2301/168849740.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  demo_data = demo_data.append(data.iloc[np.random.choice(population, sample_size, replace=False), :])
/tmp/ipykernel_2301/168849740.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  demo_data = demo_data.append(data.iloc[np.random.choice(population, sample_size, replace=False), :])
/tmp/ipykernel_2301/168849740.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  demo_data = demo_data.append(data.iloc[np.random.choice(population, sample_size, replace=False), :])
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be emp

[tensor([[ 101, 2582,  720,  ...,    0,    0,    0],
        [ 101, 3330, 1352,  ...,    0,    0,    0],
        [ 101, 4374, 2590,  ...,    0,    0,    0],
        ...,
        [ 101,  680, 6241,  ...,    0,    0,    0],
        [ 101,  123, 3299,  ...,    0,    0,    0],
        [ 101, 4220, 2111,  ...,    0,    0,    0]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([2, 2, 2, 2, 1, 1, 1, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 2, 2, 1, 2, 0, 0,
        1, 1, 0, 1, 2, 0, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0,
        0, 2, 2, 1, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 1, 2, 0, 0, 2, 0, 1, 1, 

New a pre-trained model from transformers

In [7]:
from transformers import AlbertForSequenceClassification
model = AlbertForSequenceClassification.from_pretrained(MY_PRETRAINED_MODEL, num_labels=3)
myTrainData = next(iter(myDataLoader))

Some weights of the model checkpoint at uer/albert-base-chinese-cluecorpussmall were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at uer/albert-base-chinese-cluecorpussmall and are newly initialized

In [55]:
#configurations
model.config

#number of parameters
model.num_parameters()

10550275

In [56]:
model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(21128, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [57]:
myState = model.state_dict()
myState.keys()
myState["albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight"].shape

torch.Size([768, 768])

toy example train

In [58]:
toy_example_size = 1 #只train 1 筆
model.train()
output = model(input_ids=myTrainData[0][:toy_example_size, :], token_type_ids=myTrainData[1][:toy_example_size, :], 
               attention_mask=myTrainData[2][:toy_example_size, :], labels=torch.tensor(myTrainData[3][:toy_example_size]), 
               output_hidden_states=True)
output

/tmp/ipykernel_10343/3075183845.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask=myTrainData[2][:toy_example_size, :], labels=torch.tensor(myTrainData[3][:toy_example_size]),


SequenceClassifierOutput(loss=tensor(1.1592, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0473, -0.1408,  0.1968]], grad_fn=<AddmmBackward0>), hidden_states=(tensor([[[-0.2866, -0.1826,  0.1002,  ...,  0.5232,  0.5819,  0.2270],
         [ 0.6979,  0.6134,  0.4206,  ...,  0.3448,  0.4538,  1.0133],
         [ 0.0878,  0.2886, -0.8440,  ...,  0.4203,  0.5559, -0.1841],
         ...,
         [-0.2562, -0.7522, -0.3735,  ...,  0.2546, -0.2844, -0.8386],
         [-0.1678, -0.6897, -0.3512,  ...,  0.4041, -0.4581, -0.8463],
         [-0.1055, -0.5931, -0.3233,  ...,  0.4730, -0.5742, -0.8570]]],
       grad_fn=<ViewBackward0>), tensor([[[ 0.0056, -0.2093,  0.2920,  ..., -0.2183,  0.1212, -0.2655],
         [ 0.5546, -0.2476, -0.8361,  ..., -0.2857,  0.5083, -0.4056],
         [ 0.3562,  0.6356, -1.8299,  ..., -0.1445,  0.8169, -0.8801],
         ...,
         [ 0.2381,  0.2537, -0.4637,  ...,  0.6134, -0.1090, -0.1703],
         [ 0.2353,  0.2641, -0.3853,  ...,  0.5496, -0.1490, -0.15

In [ ]:
from tqdm.cli import tqdm
def train(model, dataloader, num_epochs):
    base_lr = 1e-5
    optimizer = torch.optim.Adam([{"params": model.albert.embeddings.parameters(), "lr": base_lr},
                                  {"params": model.albert.encoder.parameters(),
                                   "lr": base_lr/2}
                                  ], lr=base_lr)
    learning_schedule = torch.optim.lr_scheduler.StepLR(optimizer=myoptimizer, step_size=3, gamma=0.1)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model.train()
    model.to(device)
    auc, total = 0, 0
    print("Start train...")
    for e in range(num_epochs):
        running_loss = 0
        for data in tqdm(dataloader):
            optimizer.zero_grad()
            labels = torch.tensor(data[3], device=device)
            output = model(input_ids=data[0].to(device),
                           token_type_ids=data[1].to(device),
                           attention_mask=data[2].to(device),
                           labels=labels,
                           output_hidden_states=False)
            output[0].backward()
            running_loss += output[0].item()
            optimizer.step()

            auc += sum(labels == torch.max(output[1], 1).indices)
            total += len(labels)
        print("[Epochs]: ", e+1, ", AUC: ", auc /
              total, ", loss: ", running_loss)
        learning_schedule.step()
    return model

Partial Layer Transfer (Fix前幾層 embedding的部分)

In [ ]:
import re
def change_layer_weights_require(model, layer_name:list=None, sign=False):
    for layer in layer_name:
        eval("model." + layer).requires_grad = sign
    return model

model = AlbertForSequenceClassification.from_pretrained(MY_PRETRAINED_MODEL, num_labels=3)
all_layer_name = list(model.state_dict().keys())
embeddings_filter = re.compile(".*\.embeddings\.")
fine_tune_layer_name = list(filter(embeddings_filter.match, all_layer_name))
print(fine_tune_layer_name)

model_fix_front_layer = change_layer_weights_require(model, fine_tune_layer_name[1:], False)
model_fix_front_layer = train(model=model_fix_front_layer, dataloader=myDataLoader)  

Partial Layer Transfer (fix後幾層）

In [ ]:
model = AlbertForSequenceClassification.from_pretrained(MY_PRETRAINED_MODEL, num_labels=3)
embeddings_filter = re.compile(".*\.encoder\.")
fine_tune_layer_name = list(filter(embeddings_filter.match, all_layer_name))
fine_tune_layer_name = list(map(lambda x:str.replace(x, ".0", "[0]"), fine_tune_layer_name))
print(fine_tune_layer_name)

model_fix_back_layer = change_layer_weights_require(model, fine_tune_layer_name[1:], False)
model_fix_back_layer = train(model=model_fix_back_layer, dataloader=myDataLoader)  

All Layer Transfer (fine-tune full model)

In [ ]:
model = AlbertForSequenceClassification.from_pretrained(MY_PRETRAINED_MODEL, num_labels=3)
model = train(model=model, dataloader=myDataLoader, num_epochs=10)

In [ ]:
#亂改
tmp = torch.nn.Sequential(
    model.pooler,
    model.pooler_activation
)
myModel = torch.nn.ModuleDict({
    "a": model.embeddings,
    "hahaha": torch.nn.Linear(128,128),
    "b": model.encoder,
    "c": torch.nn.Linear(768, 768),
    "pooler": tmp
})
output = model(input_ids=myTrainData[0], token_type_ids=myTrainData[1], 
               attention_mask=myTrainData[2], output_hidden_states=True)
len(output)